<a href="https://colab.research.google.com/github/marinaceccon1/LearningFromNetworksProject/blob/main/Degree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computation with the second approach and the degree centrality

In [1]:
#import the libraries needed
import networkx as nx
import numpy as np
import sys
import collections
import random
from builtins import sum as siumm

In [2]:
#clone the github repository containing the dataset
!git clone https://github.com/marinaceccon1/LearningFromNetworksProject.git

fatal: destination path 'LearningFromNetworksProject' already exists and is not an empty directory.


In [3]:
# load the data and create the graph representing the network
matrix = np.loadtxt('/content/LearningFromNetworksProject/out.txt', usecols=range(2))
G = nx.from_edgelist(matrix)

In [4]:
# computes the local clustering coefficient and order the dictionaries w.r.t. the keys
lcc = nx.clustering(G)
ordered_lcc = collections.OrderedDict(sorted(lcc.items()))

In [5]:
# computes the degree centrality and order the dictionaries w.r.t. the keys
deg_centralities = nx.degree_centrality(G)
ordered_deg_centralities = collections.OrderedDict(sorted(deg_centralities.items()))

In [6]:
#sort the dictionary in incresing order w.r.t. values(reverse = False)
sorted_lcc = sorted(ordered_lcc.items(), key=lambda x:x[1], reverse = False) #sorted_lcc is a list of pairs (key, value)

In [7]:
#save in len_sorted_lcc the number of values below the threshold
for i in range(len(sorted_lcc)):
  if sorted_lcc[i][1] > 0.25:
    len_sorted_lcc = i
    break

In [8]:
#sort the dictionary in decreasing order w.r.t. the values(reverse = True)
sorted_deg_centr = sorted(ordered_deg_centralities.items(),  key=lambda x:x[1], reverse = True)

In [9]:
#save in len_sorted_deg_centr the number of values above the threshold
len_sorted_deg_centr = 0
for i in range(len(sorted_deg_centr)):
  if sorted_deg_centr[i][1] < 0.00005:
    len_sorted_deg_centr = i
    break

In [10]:
#creates a list with the first len_sorted_lcc keys in the sorted_lcc list
new_sorted_lcc = []
for i in range(len_sorted_lcc):
  new_sorted_lcc.append(sorted_lcc[i][0])
s1 = set(new_sorted_lcc) #converts the list into a set, s1 eventually contains the indices of the nodes below the threshold

In [11]:
# creates a list wit the first len_sorted_deg_centr keys in sorted_deg_centr
new_sorted_bet_centr = []
for i in range(len_sorted_deg_centr):
  new_sorted_bet_centr.append(sorted_deg_centr[i][0])
s2 = set(new_sorted_bet_centr) #converts the list into a set, s2 eventually contains the indices of the nodes above the threshold

In [12]:
# computes the intersection
s = s1 & s2

In [13]:
node_list = list(s)

In [14]:
cardinalities = {} #new dictionary, the keys will be pairs of nodes that are not adjacent, the value is the cardinality 
#of the intersection of their neighbourhoods

for i in range(len(node_list)): #we consider all the nodes in this list, i.e. all the nodes with associated metrics within the thresholds
  for j in range(i+1,len(node_list)): #in order not to contemplate the same pair twice, first as (i,j) and after as (j,i), we set j>i
    
    breakout_flag = False #needed to compute a double break
    for key,value in (G.adj[node_list[i]]).items(): #check the neighbours of node_list[j]     
      if key == node_list[j]: #if inverse_sorted_ratio[j][0] is a neighbour of node_list[i]
        breakout_flag = True
        break #exit the for key,value loop
    if breakout_flag:
      break #exit the for j loop

    s1 = set(G.adj[node_list[i]]) #convert the dictionary in a set containing all the keys in the dictionary
    s2 = set(G.adj[node_list[j]])
    res = s1 & s2 #keys that belong to both sets

    cardinalities[(node_list[i], node_list[j])] = len(res) #the value associated to that pair 
    #of nodes is the cardinality of the intersection computed above

inverse_sorted_card = sorted(cardinalities.items(), key=lambda x:x[1], reverse = True) #order the dictionary by value in decreasing order

In [15]:
N = 500

In [16]:
#add the first N edges in inverse_sorted_card to a copy of the graph
G_1 = G.copy()
for i in range(N):
  u = inverse_sorted_card[i][0][0]
  v = inverse_sorted_card[i][0][1]
  G_1.add_edge(u,v)

glo_clu_coeff = nx.transitivity(G_1) #compute the global clustering coefficient of the new graph
triangles = siumm(nx.triangles(G_1).values())/3 #compute the number of triangles of the new graph
print(glo_clu_coeff) 
print(triangles)

0.14806849469334796
3518478.0
